In [1]:
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation


	removeinnerloops(g, nFE)

Remove the faces within the inner loops from sparse matrix nFE.
The return value has `g` less rows than the input `nFE`.


In [2]:
function removeinnerloops(g, nFE)
	FE = Lar.cop2lar(nFE)
	nFE = Lar.lar2cop(FE[1:end-g])
end

removeinnerloops (generic function with 1 method)

In [5]:
FE = sparse(Int8[
    1  1  1 -1  0  0  0  0  0  0  0  0;
    0  0  0  0  0  0  0  0 -1 -1 -1  1;
    -1  0  0  0  1 -1  0  0  1  0  0  0;
    0 -1  0  0  0  1 -1  0  0  1  0  0;
    0  0 -1  0  0  0  1 -1  0  0  1  0;
    0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

In [6]:
@btime removeinnerloops(2, FE)

  2.600 μs (73 allocations: 5.92 KiB)


4×12 SparseMatrixCSC{Int8, Int64} with 16 stored entries:
 1  1  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  1  1  1
 1  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  1  ⋅  ⋅

In [7]:
@code_warntype removeinnerloops(2, FE)

MethodInstance for removeinnerloops(::Int64, ::SparseMatrixCSC{Int8, Int64})
  from removeinnerloops(g, nFE) in Main at In[2]:1
Arguments
  #self#::Core.Const(removeinnerloops)
  g::Int64
  nFE@_3::SparseMatrixCSC{Int8, Int64}
Locals
  FE::Any
  nFE@_5::Any
Body::Any
1 ─       (nFE@_5 = nFE@_3)
│   %2  = Base.getproperty(Main.Lar, :cop2lar)::Any
│         (FE = (%2)(nFE@_5::SparseMatrixCSC{Int8, Int64}))
│   %4  = Base.getproperty(Main.Lar, :lar2cop)::Any
│   %5  = FE::Any
│   %6  = Base.lastindex(FE)::Any
│   %7  = (%6 - g)::Any
│   %8  = (1:%7)::Any
│   %9  = Base.getindex(%5, %8)::Any
│   %10 = (%4)(%9)::Any
│         (nFE@_5 = %10)
└──       return %10



# Ottimizzazione

In [37]:
function removeinnerloops2(g::Int64, nFE)
	FE::Vector{Vector{Int64}} = Lar.cop2lar(nFE)
	nFE::SparseMatrixCSC{Int8, Int64} = Lar.lar2cop(FE[1:end-g])
	return nFE
end

removeinnerloops2 (generic function with 1 method)

In [38]:
@btime removeinnerloops2(2, FE)

  2.622 μs (73 allocations: 5.94 KiB)


4×12 SparseMatrixCSC{Int8, Int64} with 16 stored entries:
 1  1  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  1  1  1
 1  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  1  ⋅  ⋅

In [39]:
@code_warntype removeinnerloops2(2, FE)

MethodInstance for removeinnerloops2(::Int64, ::SparseMatrixCSC{Int8, Int64})
  from removeinnerloops2(g::Int64, nFE) in Main at In[37]:1
Arguments
  #self#::Core.Const(removeinnerloops2)
  g::Int64
  nFE@_3::SparseMatrixCSC{Int8, Int64}
Locals
  FE::Vector{Vector{Int64}}
  nFE@_5::SparseMatrixCSC{Int8, Int64}
Body::SparseMatrixCSC{Int8, Int64}
1 ─       (nFE@_5 = nFE@_3)
│   %2  = Base.getproperty(Main.Lar, :cop2lar)::Any
│   %3  = (%2)(nFE@_5)::Any
│   %4  = Core.apply_type(Main.Vector, Main.Int64)::Core.Const(Vector{Int64})
│   %5  = Core.apply_type(Main.Vector, %4)::Core.Const(Vector{Vector{Int64}})
│   %6  = Base.convert(%5, %3)::Any
│         (FE = Core.typeassert(%6, %5))
│   %8  = Base.getproperty(Main.Lar, :lar2cop)::Any
│   %9  = FE::Vector{Vector{Int64}}
│   %10 = Base.lastindex(FE)::Int64
│   %11 = (%10 - g)::Int64
│   %12 = (1:%11)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│   %13 = Base.getindex(%9, %12)::Vector{Vector{Int64}}
│   %14 = (%8)(%13)::A